In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt



In [3]:
#Testing API connection and error handling
URL_count = 'https://data.melbourne.vic.gov.au/resource/b2ak-trbp.json?$limit=4370000'
try:
    response = requests.get(URL_count)
    response.raise_for_status()
    print(response)
except requests.exceptions.HTTPError as error:
    print(f'API call failed and the error msg is {error}')

URL_sensor = 'https://data.melbourne.vic.gov.au/resource/h57g-5234.json'
try:
    response2 = requests.get(URL_sensor)
    response2.raise_for_status()
    print(response2)
except requests.exceptions.HTTPError as error:
    print(f'API call failed and the error msg is {error}')

<Response [200]>
<Response [200]>


In [19]:
#Extract data from API and convert into dataframes, also check for all fields types
df_counts = pd.DataFrame.from_dict(response.json())
df_sensor = pd.DataFrame.from_dict(response2.json())
print(df_counts.dtypes)
print(df_sensor.dtypes)

id               object
date_time        object
year             object
month            object
mdate            object
day              object
time             object
sensor_id        object
sensor_name      object
hourly_counts    object
dtype: object
sensor_id             object
sensor_description    object
sensor_name           object
installation_date     object
status                object
direction_1           object
direction_2           object
latitude              object
longitude             object
location              object
note                  object
dtype: object


In [20]:
df_sensor

,sensor_id,sensor_description,sensor_name,installation_date,status,direction_1,direction_2,latitude,longitude,location,note
0,57,Bourke St Bridge,BouBri_T,2018-08-13T00:00:00.000,A,East,West,-37.81766034,144.95026189,"{'latitude': '-37.81766034', 'longitude': '144...",NaN
1,32,City Square,CSq_T,2013-12-20T00:00:00.000,R,NaN,NaN,-37.81572426,144.96686315,"{'latitude': '-37.81572426', 'longitude': '144...",Device has been removed (24/01/2017)
2,68,Flinders Ln -Degraves St (North),FLDegN_T,2020-06-03T00:00:00.000,A,East,West,-37.81683473,144.96560387,"{'latitude': '-37.81683473', 'longitude': '144...",NaN
3,40,Lonsdale St-Spring St (West),Spr201_T,2015-01-19T00:00:00.000,A,South,North,-37.80998025,144.97228184,"{'latitude': '-37.80998025', 'longitude': '144...",NaN
4,1,Bourke Street Mall (North),Bou292_T,2009-03-24T00:00:00.000,A,East,West,-37.81348124,144.96515921,"{'latitude': '-37.81348124', 'longitude': '144...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
81,75,Spring St- Flinders st (West),SprFli_T,2020-12-18T00:00:00.000,A,South,North,-37.81513959,144.97468257,"{'latitude': '-37.81513959', 'longitude': '144...",NaN
82,87,23 Errol St,Errol23_T,2022-05-20T00:00:00.000,A,South,North,-37.80453633,144.9492246,"{'latitude': '-37.80453633', 'longitude': '144...",NaN
83,72,Flinders St- ACMI,ACMI_T,2020-11-30T00:00:00.000,A,East,West,-37.81725021,144.96873406,"{'latitude': '-37.81725021', 'longitude': '144...",NaN
84,79,Flinders St (South),FliSS_T,2021-09-10T00:00:00.000,A,East,West,-37.8179273,144.96617276,"{'latitude': '-37.8179273', 'longitude': '144....",NaN


In [8]:
#Data validation and quick check on quality
df_sensor.isnull().sum()
df_counts.isnull().sum()
df_sensor.duplicated().sum()
df_counts.duplicated().sum()

0

In [5]:
#Convert field types accordingly for further aggregation and merge two dfs to get details of sensor_id
df_counts['hourly_counts']= pd.to_numeric(df_counts['hourly_counts'])
df_counts['date_time'] = pd.to_datetime(df_counts['date_time'])
df_sensor= df_sensor[['sensor_id', 'sensor_description', 'installation_date', 'status', 'note']]
df_all= df_counts.merge(df_sensor,on = 'sensor_id')
df_all['installation_date'] = pd.to_datetime(df_all['installation_date'])
df_all.head()

,id,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts,sensor_description,installation_date,status,note
0,2887628,2019-11-01 17:00:00,2019,November,1,Friday,17,34,Flinders St-Spark La,300,Flinders St-Spark La,2014-06-08,R,NaN
1,2887683,2019-11-01 18:00:00,2019,November,1,Friday,18,34,Flinders St-Spark La,240,Flinders St-Spark La,2014-06-08,R,NaN
2,2887738,2019-11-01 19:00:00,2019,November,1,Friday,19,34,Flinders St-Spark La,158,Flinders St-Spark La,2014-06-08,R,NaN
3,2887793,2019-11-01 20:00:00,2019,November,1,Friday,20,34,Flinders St-Spark La,118,Flinders St-Spark La,2014-06-08,R,NaN
4,2887848,2019-11-01 21:00:00,2019,November,1,Friday,21,34,Flinders St-Spark La,131,Flinders St-Spark La,2014-06-08,R,NaN


Calculate top 10 locations by pedestrians count by Day

In [19]:

df2 = df_all.groupby(['day','sensor_description']).agg({'hourly_counts':sum})
df2 = df2['hourly_counts'].groupby('day', group_keys=False)
result= df2.apply(lambda x: x.sort_values(ascending=False).head(10))
print(result)
result.to_csv('top10_byday.csv', index=True)


day        sensor_description               
Friday     Town Hall (West)                     24393876
           Flinders Street Station Underpass    21211433
           Melbourne Central                    19853833
           Bourke Street Mall (South)           18800045
           Princes Bridge                       18367835
                                                  ...   
Wednesday  Bourke Street Mall (North)           15043324
           Spencer St-Collins St (North)        12922557
           Flinders St-Elizabeth St (East)      12031778
           State Library                        11000867
           Flagstaff Station                    10553562
Name: hourly_counts, Length: 70, dtype: int64


Calculate top 10 locations by pedestrians count by Month

In [10]:
df3 = df_all.groupby(['month','sensor_description']).agg({'hourly_counts':sum})
df3 = df3['hourly_counts'].groupby('month', group_keys=False)
result= df3.apply(lambda x: x.sort_values(ascending=False).head(10))
print(result)
result.to_csv('top10_month.csv', index=True)

month      sensor_description               
April      Town Hall (West)                     12463968
           Melbourne Central                    10862976
           Princes Bridge                       10660422
           Flinders Street Station Underpass    10135936
           Bourke Street Mall (North)            9194871
                                                  ...   
September  Princes Bridge                        7823066
           Flinders St-Elizabeth St (East)       6435942
           State Library                         6321620
           Spencer St-Collins St (North)         6126071
           The Arts Centre                       4742644
Name: hourly_counts, Length: 120, dtype: int64


Calculate most declined location of sensor due to lockdown in last two years

In [12]:
#Taking 2019 data as benchemark, comparing with 2020 and 2021 data when several lockdowns happened,calculate % changed between 2019 and 2020, 2019 and 2021 to find out most declined location
#Formula using: (total(2019)- total(2020))/total(2019), (total(2019)-total(2021))/total(2019)

#Look for sensor_id existing in 2019,2020 and 2021 all three years

df_19 = df_all[df_all['year']== '2019']
df_2019 = df_19.groupby(['sensor_description'])['hourly_counts'].sum().reset_index()
sensor_19= df_2019['sensor_description'].values.tolist()

df_20 = df_all[df_all['year']== '2020']
df_2020 = df_20.groupby(['sensor_description'])['hourly_counts'].sum().reset_index()
sensor_20= df_2020['sensor_description'].values.tolist()

df_21 = df_all[df_all['year']== '2021']
df_2021 = df_21.groupby(['sensor_description'])['hourly_counts'].sum().reset_index()
sensor_21= df_2021['sensor_description'].values.tolist()

new_list= [df_19,df_20,df_21]
df_concat = pd.concat(new_list)

a= list(set(sensor_19).intersection(sensor_20))
sensor_all = list(set(a).intersection(sensor_21))


#Identify sensors status as "R" but removed before 2022, so they are not in scope, as it might be removed during last two years and causing wrong calculations
df_t = df_all[['date_time','sensor_description', 'status']]
df_t.date_time= pd.to_datetime(df_t.date_time).dt.strftime('%Y-%m-%d')
df_t['date_time']=pd.to_datetime(df_t['date_time'])

#Find removed sensors have volumes read before last lockdown
df_t2=df_t.drop_duplicates()
df_t2 = df_t2[df_t2['status']== 'R']
df_t2 = df_t2[df_t2['date_time'] < '2021-10-30']
df_remove =df_t2['sensor_description'].drop_duplicates()
a =list(df_remove)

#FInd removed sensors have volumes read after last lockdown
df_t3=df_t.drop_duplicates()
df_t3 = df_t3[df_t3['status']== 'R']
df_t3 = df_t3[df_t3['date_time'] > '2021-12-30']
df_remove2 =df_t3['sensor_description'].drop_duplicates()
b =list(df_remove2)

#Take left outer join from a to b to get the list of sensors removed before last lockdown which is list c
c = set(a)-set(b)


C:\Users\DollarBear\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\DOLLAR~1\AppData\Local\Temp/ipykernel_7980/1898759024.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['date_time']=pd.to_datetime(df_t['date_time'])


In [13]:
#Remove out of scope sensors
sensor_keep = set(sensor_all)-set(c)
sensor_keep

{'Alfred Place',
 'Birrarung Marr',
 'Bourke St - Spencer St (North)',
 'Bourke St Bridge',
 'Bourke St-Russell St (West)',
 'Bourke Street Mall (North)',
 'Bourke Street Mall (South)',
 'Building 80 RMIT',
 'Chinatown-Lt Bourke St (South)',
 'Chinatown-Swanston St (North)',
 'Collins Place (North)',
 'Collins Place (South)',
 'Collins Street (North)',
 'Elizabeth St-La Trobe St (East)',
 'Elizabeth St-Lonsdale St (South)',
 'Faraday St-Lygon St (West)',
 'Flinders La-Swanston St (West)',
 'Flinders St-Elizabeth St (East)',
 'Flinders Street Station Underpass',
 'Grattan St-Swanston St (West)',
 'La Trobe St (North)',
 'Lincoln-Swanston (West)',
 'Little Collins St-Swanston St (East)',
 'Lonsdale St (South)',
 'Lonsdale St - Elizabeth St (North)',
 'Lonsdale St-Spring St (West)',
 'Lygon St (East)',
 'Lygon St (West)',
 'Melbourne Central',
 'Melbourne Central-Elizabeth St (East)',
 'Melbourne Convention Exhibition Centre',
 'Monash Rd-Swanston St (West)',
 'New Quay',
 'Pelham St (Sou

In [14]:
#Calculate total counts based on 'sensor_all' list for 2019, 2020 and 2021

df_sensorall = df_concat[df_concat['sensor_description'].isin(sensor_keep)]
df_analysis = df_sensorall.groupby(['year', 'sensor_description'])['hourly_counts'].sum().reset_index()
df_analysis

,year,sensor_description,hourly_counts
0,2019,Alfred Place,2146897
1,2019,Birrarung Marr,2136024
2,2019,Bourke St - Spencer St (North),7083570
3,2019,Bourke St Bridge,7236113
4,2019,Bourke St-Russell St (West),5822623
...,...,...,...
157,2021,Tin Alley-Swanston St (West),275791
158,2021,Town Hall (West),6155040
159,2021,Victoria Point,309276
160,2021,Waterfront City,374339


In [15]:
#transform the output to be ready for % calculation
df_analysis19=df_analysis[df_analysis['year'] == '2019']
df_analysis19.rename(columns={'hourly_counts':'total_2019'},inplace=True)

df_analysis20=df_analysis[df_analysis['year'] == '2020']
df_analysis20.rename(columns={'hourly_counts':'total_2020'},inplace=True)

df_analysis21=df_analysis[df_analysis['year'] == '2021']
df_analysis21.rename(columns={'hourly_counts':'total_2021'},inplace=True)

df_tmp=df_analysis19.merge(df_analysis20, on='sensor_description')
df_analysisall= df_tmp.merge(df_analysis21, on='sensor_description')
df_analysisall= df_analysisall[['sensor_description', 'total_2019', 'total_2020', 'total_2021']]

df_analysisall

C:\Users\DollarBear\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,sensor_description,total_2019,total_2020,total_2021
0,Alfred Place,2146897,659680,461953
1,Birrarung Marr,2136024,1502283,1272681
2,Bourke St - Spencer St (North),7083570,2474396,2316080
3,Bourke St Bridge,7236113,1786890,1758182
4,Bourke St-Russell St (West),5822623,1543057,2435714
5,Bourke Street Mall (North),11637901,4791839,5080187
6,Bourke Street Mall (South),8138004,3306920,3019402
7,Building 80 RMIT,2385871,671728,2011218
8,Chinatown-Lt Bourke St (South),3703004,1146461,1356801
9,Chinatown-Swanston St (North),5186631,1995119,2288068


In [16]:
#Calculate most declined sensor location by percentage from 2019-2020
df_analysisall['%19-20']=(df_analysisall['total_2020']-df_analysisall['total_2019'])/df_analysisall['total_2019']
print(df_analysisall.sort_values(by='%19-20', ascending=True).head(1))

#Calculate most declined sensor location by percentage from 2019-2021
df_analysisall['%19-21']=(df_analysisall['total_2021']-df_analysisall['total_2019'])/df_analysisall['total_2019']
print(df_analysisall.sort_values(by='%19-21', ascending=True).head(1))

         sensor_description  total_2019  total_2020  total_2021    %19-20
21  Lincoln-Swanston (West)     1403242      316381      501972 -0.774536
        sensor_description  total_2019  total_2020  total_2021    %19-20  \
44  Southern Cross Station     6770313     1765984     1142488 -0.739158   

     %19-21  
44 -0.83125  


Calculate to get the sensor location with most growth in 2021

In [17]:

df_analysisall['%20-21']= (df_analysisall['total_2021']-df_analysisall['total_2020'])/df_analysisall['total_2020']
print(df_analysisall.sort_values(by='%20-21', ascending=False).head(1))

  sensor_description  total_2019  total_2020  total_2021    %19-20   %19-21  \
7   Building 80 RMIT     2385871      671728     2011218 -0.718456 -0.15703   

     %20-21  
7  1.994096  
